## **Working with Geospatial Data – Session 5**  

This session explores geospatial data analysis using GeoPandas.
The notebook specifically focuses on urban data processing in New York City (NYC), analyzing:

Schools and Subway Entrances 📍
Bike Paths and Parks 🚲 🌳
Neighborhood Boundaries 🏙

In [1]:
import geopandas as gpd

In [2]:
schools = gpd.read_file("../data/nyc/nyc_school_points/SchoolPoints_APS_2024_08_28.shp")
subways = gpd.read_file("../data/nyc/nyc_subway_entrances/nyc_subway_entrances.shp")
bike_paths = gpd.read_file("../data/nyc/nyc_bike_routes_20241223.geojson")
neighborhoods = gpd.read_file("https://raw.githubusercontent.com/HodgesWardElliott/custom-nyc-neighborhoods/refs/heads/master/custom-pedia-cities-nyc-Mar2018.geojson")
parks = gpd.read_file("../data/nyc/Parks Properties_20241223.geojson")

In [3]:
schools = schools.to_crs("EPSG:3857")
subways = subways.to_crs("EPSG:3857")
bike_paths = bike_paths.to_crs("EPSG:3857")
neighborhoods = neighborhoods.to_crs("EPSG:3857")
parks = parks.to_crs("EPSG:3857")

In [4]:
sindex = neighborhoods.sindex

In [5]:
schools_in_neighborhoods = gpd.sjoin(schools, neighborhoods, how="inner", predicate="within")

In [6]:
schools_in_neighborhoods.head()

,ATS,Building_C,Location_C,Name,Geographic,Latitude,Longitude,geometry,index_right,neighborhood,boroughCode,borough,X.id
0,01M015,M015,M015,P.S. 015 Roberto Clemente,1,40.722075,-73.978747,POINT (-8235276.446 4971433.816),86,East Village,1,Manhattan,http://nyc.pediacities.com/Resource/Neighborho...
1,01M020,M020,M020,P.S. 020 Anna Silver,1,40.721305,-73.986312,POINT (-8236118.578 4971320.718),176,Lower East Side,1,Manhattan,http://nyc.pediacities.com/Resource/Neighborho...
2,01M034,M034,M034,P.S. 034 Franklin D. Roosevelt,1,40.726008,-73.975058,POINT (-8234865.788 4972011.521),86,East Village,1,Manhattan,http://nyc.pediacities.com/Resource/Neighborho...
3,01M063,M063,M063,The STAR Academy - P.S.63,1,40.724440,-73.986214,POINT (-8236107.668 4971781.199),86,East Village,1,Manhattan,http://nyc.pediacities.com/Resource/Neighborho...
4,01M064,M064,M064,P.S. 064 Robert Simon,1,40.723130,-73.981597,POINT (-8235593.706 4971588.778),86,East Village,1,Manhattan,http://nyc.pediacities.com/Resource/Neighborho...


In [7]:
# Aggregate the results: count schools per neighborhood and include geometry

schools_count_by_neighborhood = schools_in_neighborhoods.groupby('neighborhood').agg(
    school_count=('index_right', 'count')
).reset_index()

In [8]:
schools_count_by_neighborhood

,neighborhood,school_count
0,Allerton,12
1,Arden Heights,1
2,Arrochar,2
3,Arverne,2
4,Astoria,14
...,...,...
211,Windsor Terrace,7
212,Woodhaven,3
213,Woodlawn,1
214,Woodrow,1


In [9]:
schools_count_by_neighborhood = neighborhoods.merge(
    schools_count_by_neighborhood, on='neighborhood', how='left'
).fillna({'school_count': 0})


In [10]:
schools_count_by_neighborhood

,neighborhood,boroughCode,borough,X.id,geometry,school_count
0,Allerton,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8220788.214 4993431.406, -8220479.3...",12.0
1,Alley Pond Park,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8209070.244 4973902.938, -8209112.6...",0.0
2,Arden Heights,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8256547.374 4947814.753, -8256546.8...",1.0
3,Arlington,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8255425.402 4959593.695, -8255451.0...",0.0
4,Arrochar,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8244408.305 4952520.907, -8244409.6...",2.0
...,...,...,...,...,...,...
307,Woodside,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8226606.801 4976664.013, -8226525.9...",5.0
308,East Williamsburg,3,Brooklyn,None,"POLYGON ((-8231107.865 4971393.424, -8231029.9...",5.0
309,Hamilton Heights,1,Manhattan,None,"POLYGON ((-8232132.004 4988019.867, -8231085.6...",10.0
310,Williamsburg,3,Brooklyn,None,"POLYGON ((-8232919.255 4971877.704, -8232410.0...",28.0


In [11]:
schools_count_by_neighborhood = gpd.GeoDataFrame(schools_count_by_neighborhood, geometry='geometry', crs=neighborhoods.crs)


In [12]:
schools_count_by_neighborhood

,neighborhood,boroughCode,borough,X.id,geometry,school_count
0,Allerton,2,Bronx,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8220788.214 4993431.406, -8220479.3...",12.0
1,Alley Pond Park,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8209070.244 4973902.938, -8209112.6...",0.0
2,Arden Heights,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8256547.374 4947814.753, -8256546.8...",1.0
3,Arlington,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8255425.402 4959593.695, -8255451.0...",0.0
4,Arrochar,5,Staten Island,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8244408.305 4952520.907, -8244409.6...",2.0
...,...,...,...,...,...,...
307,Woodside,4,Queens,http://nyc.pediacities.com/Resource/Neighborho...,"POLYGON ((-8226606.801 4976664.013, -8226525.9...",5.0
308,East Williamsburg,3,Brooklyn,None,"POLYGON ((-8231107.865 4971393.424, -8231029.9...",5.0
309,Hamilton Heights,1,Manhattan,None,"POLYGON ((-8232132.004 4988019.867, -8231085.6...",10.0
310,Williamsburg,3,Brooklyn,None,"POLYGON ((-8232919.255 4971877.704, -8232410.0...",28.0


In [13]:
schools_count_by_neighborhood.head(10).explore()